# Random Network Distillation 

In [1]:
import os
import gymnasium as gym
from gym.wrappers import RecordVideo
from IPython.display import Video, display, clear_output
from tqdm import tqdm
import torch 
# torch default device
if  torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
# device = torch.device("mps")
torch.set_default_device(device)
print(f"Using device: {device}")

from rnd_rl.runner.policy_runner import PPOConfig, PolicyRunner

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


Using device: cuda


/home/jkamohara3/DRL/DRL_RND/drl_env/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/jkamohara3/DRL/DRL_RND/drl_env/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened

In [2]:
# @title Visualization code. Used later.

def visualize(agent):

    video_dir = "./videos"  # Directory to save videos
    os.makedirs(video_dir, exist_ok=True)

    # Create environment with proper render_mode
    env = gym.make("InvertedPendulum-v5", render_mode="rgb_array", reset_noise_scale=0.2)

    # Apply video recording wrapper
    env = RecordVideo(env, video_folder=video_dir, episode_trigger=lambda x: True)

    obs, _ = env.reset()


    for t in range(4096):
        actions, _ = agent.get_action(torch.Tensor(obs)[None, :].to(device))
        obs, _, done, _, _ = env.step(actions.squeeze(0).cpu().numpy())

        if done:
            # self.writer.add_scalar("Duration", t, i)
            break

    env.close()

    # Display the latest video
    video_path = os.path.join(video_dir, sorted(os.listdir(video_dir))[-1])  # Get the latest video


    clear_output(wait=True)
    display(Video(video_path, embed=True))

In [4]:
n_envs = 64
envs = gym.vector.SyncVectorEnv(
    [lambda: gym.make("InvertedPendulum-v5", reset_noise_scale=0.2) for _ in range(n_envs)]
    )

### PPO baseline

In [5]:
ppo_cfg = PPOConfig(
    use_rnd=False, 
    clip_params=0.2,
    init_noise_std=1.0, 
)

In [6]:
num_epochs = 250
policy_runner = PolicyRunner(
    envs=envs, 
    policy_cfg=ppo_cfg, 
    num_mini_epochs=10, 
    device=device, 
    experiment_name="PPO"
)
for epoch in tqdm(range(num_epochs)):
    policy_runner.rollout(epoch)
    policy_runner.update()

wandb: Currently logged in as: jnskkmr to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


100%|██████████| 250/250 [01:38<00:00,  2.54it/s]


In [7]:
visualize(policy_runner.alg)
print("PPO trained agent")

PPO trained agent


### PPO with RND

In [8]:
ppo_rnd_cfg = PPOConfig(
    use_rnd=True, 
    clip_params=0.2,
    init_noise_std=1.0, 
)

In [9]:
num_epochs = 250 
rnd_policy_runner = PolicyRunner(
    envs=envs, 
    policy_cfg=ppo_rnd_cfg, 
    num_mini_epochs=10,
    device=device,
    experiment_name="PPO_RND"
)
for epoch in tqdm(range(num_epochs)):
    rnd_policy_runner.rollout(epoch)
    rnd_policy_runner.update()

Extrinsic Reward,▁▃██████████████████████████████████████
Reward,▁▅▇█████████████████████████████████████
Extrinsic Reward,1
Reward,1


100%|██████████| 250/250 [01:52<00:00,  2.22it/s]


In [10]:
visualize(rnd_policy_runner.alg)
print("RND PPO trained agent")

RND PPO trained agent


### Reward normalization only

In [ ]:

ppo_rnd_reward_normalization_cfg = PPOConfig(
    use_rnd=True, 
    clip_params=0.2,
    init_noise_std=1.0, 
    reward_normalization = True
)


In [ ]:
num_epochs = 250 
rnd_reward_norm_policy_runner = PolicyRunner(
    envs=envs, 
    policy_cfg=ppo_rnd_reward_normalization_cfg, 
    num_mini_epochs=10,
    device=device, 
    experiment_name="PPO_RND_normalization"
)
for epoch in tqdm(range(num_epochs)):
    rnd_reward_norm_policy_runner.rollout(epoch)
    rnd_reward_norm_policy_runner.update()

In [ ]:
visualize(rnd_reward_norm_policy_runner.alg)
print("RND PPO trained agent with reward normalization")

### Reward and observation normalization

In [11]:
ppo_rnd_all_normalization_cfg = PPOConfig(
    use_rnd=True, 
    clip_params=0.2,
    init_noise_std=1.0, 
    reward_normalization = True,
    obs_normalization = True
)

In [12]:
num_epochs = 250 
rnd_all_norm_policy_runner = PolicyRunner(
    envs=envs, 
    policy_cfg=ppo_rnd_all_normalization_cfg, 
    num_mini_epochs=10,
    device=device, 
    experiment_name="PPO_RND_normalization"
)
for epoch in tqdm(range(num_epochs)):
    rnd_all_norm_policy_runner.rollout(epoch)
    rnd_all_norm_policy_runner.update()

Extrinsic Reward,▁▃▅█████████████████████████████████████
Reward,█▆▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Extrinsic Reward,1
Reward,1.05537


100%|██████████| 250/250 [02:18<00:00,  1.81it/s]


In [13]:
visualize(rnd_all_norm_policy_runner.alg)
print("RND PPO trained agent with observation normalization")

RND PPO trained agent with observation normalization
